# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#    Need for slowing down the query loop and date stamps
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
weather_data_o112 = pd.read_csv("big_weather_data_df.csv")

#works
#weather_data_o112 = pd.read_csv("../output_data/com_weather_data_df.csv")
#weather_data_df = weather_data_o112
#weather_data_o112


In [46]:
#time stamp
epoch = (time.time())

# converts epoch into something useful
#time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(epoch))
time_stamp = time.strftime("%d %b %Y %H:%M:%S", time.localtime(epoch))
time_stamp

'20 Oct 2020 13:34:31'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
 # Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

#extra import probably delete
# Fill NaN values and convert to float
humid = weather_data_df["Humidity (RH)%"].astype(float)

In [48]:
gmaps.configure(api_key = g_key)

In [49]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0],zoom_level = 2)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid , 
                                 dissipating=False, max_intensity=85,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

#label map


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * Drop any rows will null values.

    * A max temperature lower than 80 degrees but higher than 70.
    * Wind speed less than 10 mph.
    * Zero cloudiness.
    * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [51]:
weather_data_o112

,Unnamed: 0,City ID,confirm name,Confirm Country,Latitude,Longitude,Temp Kelvin,Temp C,Temp F,Humidity (RH)%,Cloud% Cover,wind speed (m/s),wind speed (mph),Time stamp
0,0,2353099,Abakaliki,NG,6.32,8.12,303.18,30.0,86.0,73,48,1.65,3.67,['20 Oct 2020 12:23:44']
1,1,110690,Abhā,SA,18.22,42.51,293.15,20.0,68.0,10,1,5.10,11.33,['20 Oct 2020 12:23:44']
2,2,2324433,Abonnema,NG,4.71,6.79,301.15,28.0,82.0,78,75,2.60,5.78,['20 Oct 2020 12:23:44']
3,3,172515,Abu Samrah,SY,35.30,37.18,297.13,24.0,75.0,28,0,3.90,8.67,['20 Oct 2020 12:23:44']
4,4,3408368,Acaraú,BR,-2.89,-40.12,304.66,32.0,89.0,51,49,10.20,22.67,['20 Oct 2020 12:23:44']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,597,2220853,Yokadouma,CM,3.52,15.05,296.35,23.0,74.0,84,23,0.59,1.31,['20 Oct 2020 12:23:44']
598,598,6355222,Yulara,AU,-25.24,130.99,295.15,22.0,72.0,56,0,3.60,8.00,['20 Oct 2020 12:23:44']
599,599,2317982,Yuli,NG,9.70,10.27,301.90,29.0,84.0,47,40,1.67,3.71,['20 Oct 2020 12:23:44']
600,600,464790,Zapolyarnyy,RU,69.42,30.81,274.15,1.0,34.0,98,90,5.10,11.33,['20 Oct 2020 12:23:44']


In [74]:
mod_temp = weather_data_df.loc[(weather_data_df["Temp F"] < 80) & (weather_data_df["Temp F"] > 70) & (weather_data_df["wind speed (mph)"] < 10) & (weather_data_df["Cloud% Cover"] == 0)].copy()

In [75]:
mod_temp

,Unnamed: 0,City ID,City,confirm name,Country Code,Confirm Country,Latitude,Longitude,Temp Kelvin,Temp C,Temp F,Humidity (RH)%,Cloud% Cover,wind speed (m/s),wind speed (mph)
6,7,2324433,Abonnema,Abonnema,NG,NG,4.71,6.79,297.97,25,77,92,0,1.87,4.16
11,12,153709,Mwandiga,Mwandiga,TZ,TZ,-4.83,29.66,296.93,24,75,60,0,2.19,4.87
52,56,933995,Souillac,Souillac,MU,MU,-20.52,57.52,298.71,26,78,62,0,4.47,9.93
66,71,3985710,Cabo San Lucas,Cabo San Lucas,MX,MX,22.89,-109.91,296.94,24,75,40,0,2.24,4.98
69,75,4726521,San Patricio,San Patricio,MX,US,28.02,-97.52,298.52,26,78,97,0,3.68,8.18
79,88,934322,Mahebourg,Mahébourg,MU,MU,-20.41,57.70,298.71,26,78,64,0,3.13,6.96


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
 # set up additional columns to hold information
mod_temp["Hotel Name"] = ""
mod_temp["Hotel Address"] = ""
mod_temp["Hotel Rating"] = ""
mod_temp.head()

,Unnamed: 0,City ID,City,confirm name,Country Code,Confirm Country,Latitude,Longitude,Temp Kelvin,Temp C,Temp F,Humidity (RH)%,Cloud% Cover,wind speed (m/s),wind speed (mph),Hotel Name,Hotel Address,Hotel Rating
6,7,2324433,Abonnema,Abonnema,NG,NG,4.71,6.79,297.97,25,77,92,0,1.87,4.16,,,
11,12,153709,Mwandiga,Mwandiga,TZ,TZ,-4.83,29.66,296.93,24,75,60,0,2.19,4.87,,,
52,56,933995,Souillac,Souillac,MU,MU,-20.52,57.52,298.71,26,78,62,0,4.47,9.93,,,
66,71,3985710,Cabo San Lucas,Cabo San Lucas,MX,MX,22.89,-109.91,296.94,24,75,40,0,2.24,4.98,,,
69,75,4726521,San Patricio,San Patricio,MX,US,28.02,-97.52,298.52,26,78,97,0,3.68,8.18,,,


In [120]:
# find the closest restaurant of each type to coordinates

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    #"keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in mod_temp.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
      

########     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#   print(name_address.url)
 #########   # convert to json
#    name_address = name_address.json()
#    print(json.dumps(name_address, indent=4, sort_keys=True))

#name_address

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        mod_temp.loc[index,"Hotel Name"] = name_address["results"][0]["name"]
        #mod_temp.loc[index,"Hotel Address"] = name_address["results"][0]["vicinity"]
        #mod_temp.loc[index,"Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

TypeError: 'Response' object is not subscriptable

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
